In [ ]:
from IPython.display import display, HTML

import pandas as pd
import matplotlib.pyplot as plt

from src.database import read_sql
from src.plotting import plot_multi

In [ ]:
# Schichtfunktionen
ignored_shifts = [
# from sickness shifts
    'K>6', # länger 6 wochen krank -> nicht inzidenz
    'KK', # kind krank -> nicht inzidenz
# from absent non-sick
    # '!', # urlaub, hier meldet man sich trotzdem krank
    'ABW-Sonstige', # sonstige sind zu vielfältig
    'Sonstige',
    # 'AH', # einsatz außer haus
    'AM', # Abwesenheitsmeldung, würden sich sonst krank melden
    'AWU', # arbeits-/wegeunfall -> werden nicht von maßnahmen beeinflusst
    # 'A-ZG', # ausgleich zeitguthaben, betrachtet wie urlaub
    # 'BR', # betriebsratstag, würden sich auch krank melden
    'BV', # beschäftigungsverbot, beeinflussen inzidenz nicht
    # 'DR', # dienstreise, würden sich auch krank melden
    'EU-Re', # erwerbsunfähigkeitsrente, i.e. > 1.5J abwesend
    'EZ', # elternzeit, lange nicht da
    # 'F', # fortbildung, würden sich krank melden
    # 'KA', # kurzarbeit, würden sich auch krank melden
    'KUR', # kur, länger nicht da
    'MU', # mutterschutz, länger nicht da
    'QT', # quarantäne
    # 'R', # regenerationstag, würde sich auch krank melden
    # 'S', # streik, sonderurlaub, würden sich auch krank melden
    # 'U', # urlaub, würden sich krank melden
    # 'WB', # weiterbildung, würden sich krank melden
    # 'X', # dienstfrei, würden sich auch krank melden; machen sie aber nicht -> WEIHNACHTSKNICKE
    # 'X-F', # ersatzruhetag, würden sich auch krank melden
    # 'X-S' # ersatzruhetag, würden sich auch krank melden
]
sql_ignored_shifts = ' AND '.join((f"neu_kurz NOT ILIKE('{shift}')" for shift in ignored_shifts))
sql_sickness_abbr = f'''
    SELECT neu_kurz as abbr, string_agg(sdienst_lang, ', ') as description FROM spxdb_archiv_2023.sdienst_mapping
    WHERE sdienst_lang ILIKE('%krank%') AND sdienst_lang NOT ILIKE('%nicht krank%') AND {sql_ignored_shifts}
    GROUP BY abbr
'''
print('Schichtfunktionen mit Krankheit')
display(read_sql(sql_sickness_abbr))

sql_all_abbr = f'''
    SELECT neu_kurz as abbr, string_agg(sdienst_lang, ', ') as description FROM spxdb_archiv_2023.sdienst_mapping
    WHERE {sql_ignored_shifts}
    GROUP BY abbr
'''
print('Nicht abwesende und kranke Schichten, i.e. alle die für Inzidenz relevant sind (erste 10)')
display(read_sql(sql_all_abbr + ' LIMIT 5'))

In [ ]:
# Schichten
def _sql_shifts_from_abbr(abbr: str) -> str:
    return f'''WITH mapping AS ({abbr})
        SELECT *
        FROM mapping JOIN spxdb_archiv_2023.monatsplan_adapted AS plan
        ON mapping.abbr = plan.sdienst_kurz_adapted
        WHERE plan.mpebene = 1 -- filter for "ist" (vs. e.g. "soll")
    '''
sql_sickness_plan = _sql_shifts_from_abbr(sql_sickness_abbr)
print("Schichten mit Krankheit")
display(read_sql(sql_sickness_plan + ' LIMIT 3'))

sql_all_plan = _sql_shifts_from_abbr(sql_all_abbr)
print("Alle für Inzidenz zu zählende Schichten")
display(read_sql(sql_all_plan + ' LIMIT 3'))

In [ ]:
def _counts(plan: str, name: str) -> pd.DataFrame:
    return read_sql(f'''
        WITH plan AS ({plan})
        SELECT
            DATE_TRUNC('week', (DATE '1990-01-01' + (tag_f * INTERVAL '1 day'))) AS "Kalenderwoche",
            COUNT(*) as "{name}"
         FROM plan GROUP BY "Kalenderwoche";
    ''')

sickness_count = _counts(sql_sickness_plan, 'sick')
all_count = _counts(sql_all_plan, 'all')

evb = pd.merge(sickness_count, all_count, on='Kalenderwoche')
evb['EVB'] = evb['sick'] / evb['all'] * 100000
evb = evb.drop(columns=['sick', 'all'])

plt.figure(figsize=(10, 5))
plt.plot(evb['Kalenderwoche'], evb['EVB'])
plt.title('7-Tage Inzidenz im EVB')
plt.xlabel('Kalenderwoche')
plt.ylabel('Anzahl')

In [ ]:
# load grippeweb data & parse dates
grippeweb_original = pd.read_csv('https://raw.githubusercontent.com/robert-koch-institut/GrippeWeb_Daten_des_Wochenberichts/main/GrippeWeb_Daten_des_Wochenberichts.tsv',
sep='\t', index_col='Kalenderwoche')
grippeweb_original.index = pd.to_datetime(grippeweb_original.index + '-1', format='%G-W%V-%u')

# filter & accumulate
grippeweb = grippeweb_original.loc[
     grippeweb_original.Altersgruppe.str.contains('15-34|35-59') & # Region 'Osten' only has 00+ age group
    (grippeweb_original.Region == 'Bundesweit') &
    (grippeweb_original.Erkrankung == 'ARE')
]\
    .groupby('Kalenderwoche')\
    .Inzidenz.mean()\
    .reset_index()\
    .rename(columns={ 'Inzidenz': 'GrippeWeb' })


# load consultations data & parse dates
consultation_original = pd.read_csv('https://raw.githubusercontent.com/robert-koch-institut/ARE-Konsultationsinzidenz/main/ARE-Konsultationsinzidenz.tsv',
sep='\t', index_col='Kalenderwoche')
consultation_original.index = pd.to_datetime(consultation_original.index + '-1', format='%G-W%V-%u')

# filter & accumulate
consultations = consultation_original.loc[
    consultation_original.Altersgruppe.str.contains('15-34|35-59') &
    consultation_original.Bundesland.str.contains('Bundesweit')
]\
    .groupby('Kalenderwoche')\
    .ARE_Konsultationsinzidenz.mean()\
    .reset_index()\
    .rename(columns={ 'ARE_Konsultationsinzidenz': 'Konsultationen' })

In [ ]:
sickl = read_sql('''
    SELECT
        DATE_TRUNC('week', DATE(ks.min_date)) AS "Kalenderwoche",
        SUM(ks.anzahl) as "Krankschreibungen"
    FROM
        student_data.mvz_krankschreibungen as ks
    GROUP BY "Kalenderwoche"
    ORDER BY "Kalenderwoche"
''').sort_values('Kalenderwoche').set_index('Kalenderwoche')
sickl.plot()
sickl.index = pd.to_datetime(sickl.index).tz_localize(None)

In [ ]:
sickl.head()

In [ ]:
# merge data
merged = pd.merge(
    pd.merge(
        evb,
        consultations,
        on='Kalenderwoche', how='outer'),
    grippeweb,
    on='Kalenderwoche', how='outer'
).sort_values('Kalenderwoche').set_index('Kalenderwoche')
merged.head()

In [ ]:
def _plot_since(date: str):
    ax = merged.loc[pd.to_datetime(date):].plot(
        alpha=0.6,
        figsize=(10, 5),
        title=f'Inzidenz-Vergleich seit {date}',
        xlabel='Kalenderwoche',
        ylabel='7-Tage Inzidenz'
    )
    ax.legend(loc='upper left')

    from pandas.plotting._matplotlib.style import get_standard_colors
    ax2 = ax.twinx()
    sickl.loc[pd.to_datetime(date):].plot(
        ax=ax2,
        color=get_standard_colors(num_colors=len(merged.columns) + 1)[-1],
        alpha=0.6
    )
    
_plot_since('2014')
_plot_since('2020')

In [ ]:
def plot_winter(season: int):
    df =  merged.loc[pd.to_datetime(f'20{season}-10-01'):pd.to_datetime(f'20{season + 1}-03-31')]
    ax = df.pct_change().plot.bar(
        alpha=0.7,
        figsize=(10, 3),
        title=f'Vergleich der relativen Änderung der Inzidenz im Winter 20{season}/{season + 1}',
        rot=0,
        ylabel='Relative Änderung 7-Tage Inzidenz'
    )
    tick_labels = ['' if d.day > 7 else d.strftime('%b') for d in df.index]
    ax.set_xticklabels(tick_labels)
    ax.legend(loc='upper right')
    
    ax = df.plot(
        alpha=0.7,
        figsize=(10, 3),
        title=f'Inzidenz-Vergleich im Winter 20{season}/{season + 1}',
        ylabel='7-Tage Inzidenz'
    )
    ax.legend(loc='upper right')
    
    df = (df - df.min()) / (df.max() - df.min())
    ax = df.plot(
        alpha=0.7,
        figsize=(10, 3),
        title=f'Min-Max-normalisierte 7-Tage Inzidenz im Winter 20{season}/{season + 1}',
        ylabel='Normalisierte Inzidenz'
    )
    ax.legend(loc='upper right')

In [ ]:
plot_winter(20)

In [ ]:
plot_winter(21)

In [ ]:
plot_winter(22)

In [ ]:
plot_winter(17)